In [2]:
import os
from typing import List, Dict
from langchain_community.document_loaders import PyPDFLoader

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 0단계: 문서 로드 -----------------------------------------------------------
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = "../data/콘텐츠분쟁해결_사례.pdf"
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

In [4]:
# 1단계: 문서 분할 설정 -------------------------------------------------------
# LangChain 버전에 따라 모듈 경로가 다를 수 있어 호환 처리
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,      # 법률 사례는 1200~1800자 권장
    chunk_overlap=300,    # 맥락 보존 200~400자
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ],
)
splits = text_splitter.split_documents(documents)


In [5]:
# 2단계: 임베딩 모델 설정 -----------------------------------------------------
# 환경변수 UPSTAGE_API_KEY 필요
#  - Windows PowerShell:  $env:UPSTAGE_API_KEY="YOUR_UPSTAGE_API_KEY"
#  - macOS/Linux:         export UPSTAGE_API_KEY="YOUR_UPSTAGE_API_KEY"
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# 3단계: 벡터스토어 & 검색기 설정 -------------------------------------------
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(splits, embeddings)

retriever = vectorstore.as_retriever(
    search_type="similarity",      # 다양성 필요하면 "mmr"
    search_kwargs={"k": 5}
)

In [6]:
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

In [7]:
from langchain.prompts import PromptTemplate
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1) 제시된 사례들을 근거로 답변하세요.
2) 관련 법령이나 조항이 있다면 명시하세요.
3) 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요.
4) 실무적 해결방안을 단계별로 제시하세요.
5) 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요.

전문 법률 조언:"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
)

# 6단계: QA 체인 생성 ---------------------------------------------------------
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                       # 위에서 설정한 LLM
    chain_type="stuff",            # 문서들을 하나로 합쳐 처리
    retriever=retriever,           # 검색기
    chain_type_kwargs={"prompt": prompt},  # 법률 자문 프롬프트
    return_source_documents=True   # 참조 문서 반환
)


In [8]:
test_questions: List[str] = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

In [9]:
def run_once(question: str) -> Dict:
    """
    질의 1건 수행하고 답변과 소스(상위 k개)를 함께 반환
    RetrievalQA의 입력 키는 기본 'query'
    """
    result = qa_chain({"query": question})
    answer = result.get("result", "")
    source_docs = result.get("source_documents", []) or []

    # 소스 요약 구성 (페이지/스니펫)
    sources: List[Dict] = []
    for i, d in enumerate(source_docs, start=1):
        page = d.metadata.get("page")
        page_tag = "p.{}".format(page + 1) if isinstance(page, int) else "p.?"
        content = d.page_content or ""
        snippet = content[:160] + "…" if len(content) > 160 else content
        sources.append({"rank": i, "page": page_tag, "snippet": snippet})
    return {"question": question, "answer": answer, "sources": sources}


In [12]:
print("=== 콘텐츠분쟁해결 RAG 테스트 ===\n")
for q in test_questions:
    res = run_once(q)
    print(f"[Q] {res['question']}\n")
    print(f"[A]\n{res['answer']}\n")
    print("[참조 조각]")
    for s in res["sources"]:
        print("  - #{} ({}): {}".format(s["rank"], s["page"], s["snippet"]))
    print("\n" + "=" * 80 + "\n")

=== 콘텐츠분쟁해결 RAG 테스트 ===

[Q] 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[A]
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사례 해결 방안  

#### 1. **사례 분석을 통한 법적 쟁점 정리**  
제시된 사례들을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음 요소에 따라 해결 가능성이 달라집니다.  
- **(1) 오류 발생 사실 확인**: 게임사의 시스템 오류가 객관적으로 입증되어야 합니다.  
  - *사례 2009년 9월 5일 사건*에서는 신청인의 주장 외에 다른 이용자의 유사 피해 신고가 없어 오류 인정이 어려웠습니다.  
- **(2) 계정 소유권 확인**: 신청인이 계정 명의자여야 복구 요청이 가능합니다.  
  - *사례 2006년 시스템 오류 사건*에서는 타인 명의의 계정 사용 시 약관 위반으로 복구가 거절되었습니다.  
- **(3) 게임사의 약관 및 대응**:  
  - *사례 2006년 프로그램 오류 사건*에서는 게임사가 오류를 인정하고 복구한 반면,  
  - *사례 2007년 해킹 아이템 회수 사건*에서는 약관 제11조(저작권과 아이템 소유권)를 근거로 회수 조치를 정당화했습니다.  

#### 2. **관련 법령 및 약관 검토**  
- **민법 제250조(도품·유실물 특례)**:  
  - 게임 아이템이 "금전"에 준하는 디지털 자산으로 볼 경우, 반환 청구가 제한될 수 있습니다.  
  - 다만, *사례 2007년 사건*에서 게임머니(딜)는 환급되었으나, 아이템은 저작권법상 게임사의 소유로 인정되어 반환되지 않았습니다.  
- **게임산업진흥에 관한 법률 제12조(준용규정)**:  
  - 게임사는 이용약관을 통해 서비스 이용 조건을 명시해야 하며, 약관상 "시스템 오류 시 복구 불가" 조항이 있다면 이를 근거로 거부할 수 있습니다.  
- **전자상거래 등에서의 소비자보호에 관한 법률 제